## Определение принадлежности текстов к заданной тематике

Нужно определять схожесть текста вакансии с заданной тематикой.

Имея набор текстов определенной базовой тематики и набор текстов неизвестной тематики определить, относятся ли тексты к заданной тематике или нет. Предложен список id текстов (test_***.txt текстов), по которым нужно подготовить ответ.

Схожесть вакансий может использоваться в рамках content-based рекомендательной системы.

Файлы двух типов:

файлы base_*.txt - файлы с текстом одинаковой заданной тематики;  
файлы test_*.txt - файлы с текстом неизвестной тематики.

Содержимым всех файлов является текст (описание вакансии), содержащий html-теги.

In [2]:
import glob
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pymorphy2
from tqdm.auto import tqdm
from nltk.corpus import words, stopwords
from bs4 import BeautifulSoup 
from pymystem3 import Mystem
from string import punctuation
import json


#### Cчитывание base  и test файлов в отдельные датафреймы

In [3]:
dir_input = '/Users/loban/Projects/Newprolab/ml/lab04/lab04data/base*.txt'
files = glob.glob(dir_input)
base_df = pd.concat([pd.read_csv(f,header=None,sep='\t') for f in files],ignore_index=True)

bases_index = []

for file in files:
    temp = str(file.split("\\")[1].split('.')[0])
    bases_index.append(temp)
    
base_df.index = bases_index

In [4]:
dir_input = '/Users/loban/Projects/Newprolab/ml/lab04/lab04data/test*.txt'
files = glob.glob(dir_input)
test_df = pd.concat([pd.read_csv(f,header=None,sep='\t') for f in files],ignore_index=True)

test_index = []

for file in files:
    temp = str(file.split("\\")[1].split('.')[0])
    test_index.append(temp)
    
test_df.index = test_index

In [5]:
# Тестовые + бейз тексты вакансий

all_df = pd.concat([test_df, base_df])

In [6]:
all_df.head(5)

,0
test_1,<p><strong>В наши магазины приходят за красивы...
test_10,"<p>АО «ПанКлуб», представляющий всемирно призн..."
test_100,"<p>Что нужно энергичным людям? Интересная, выс..."
test_1000,<p><strong>Требования:</strong></p> <ul> <li>П...
test_1001,"<p>Что нужно энергичным людям? Интересная, выс..."


### Создание функции для обработки текста

In [7]:
def words_only(text):
    text = BeautifulSoup(text).get_text()      
    words = re.sub("[^а-яА-яa-zA-z]", " ", text).lower().split()                                
    stopw = set(stopwords.words("russian"))                  
    cleaned_words = [w for w in words if not w in stopw]  
    return(" ".join(cleaned_words))


In [8]:
# Доп.предобработка - создание лемматайзера

mystem = Mystem() 

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    text = " ".join(tokens)
    return text


### Обработка текстов

In [9]:
texts_count = all_df[0].size
clean_texts = []

for i in range(0, texts_count):
    clean_texts.append(words_only(all_df[0][i]))
    

In [10]:
clean_texts_s2 = []

for i in tqdm(clean_texts):
    clean_texts_s2.append(preprocess_text(i))
    

In [11]:
clean_texts_s2[0]

'наш   магазин   приходить   красивый   удобный   предмет   кухня   столовая   помогать   наш   покупатель   превращать   приготовление   еда   увлекательный   процесс   наш   ожидание   кандидат   готовность   зарабатывать   понимание   ключевой   показатель   эффективность   продажа   коммуникабельность   позитивность   обучаемость   быстрота   интерес   кулинария   предлагать   стабильный   оклад   плюс   возможность   зарабатывать   высокий   премия   бонус   продажа   удобный   график   работа   плавать   выходной   дополнительный   привилегия   тот   долго   мы   работать   возможность   работать   самый   хороший   оригинальный   товар   кухня   столовая   работа   магазин   уникальный   атмосфера   возможность   развиваться   делать   карьера \n'

### Создание векторизированного датафрейма каждого текста с помощью CountVectorizer

In [12]:
vectorizer = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, 
                             stop_words=None, max_features=20000) 

sparse_matrix = vectorizer.fit_transform(clean_texts_s2)

cleaned_matrix = sparse_matrix.todense()


In [13]:
cleaned_matrix


matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
tests = cleaned_matrix[:-20]
bases = cleaned_matrix[-20:]


### Расчет косинусной меры сходства обработанных векторов (мера сходства)

In [15]:
# Мера рассчитывается как сходства каждого тестового текста с 20-ю базовыми текстами и далее суммируется

res_df = pd.DataFrame(columns=['CosMetric'])

for i in tqdm(range(len(tests))):
    our_sum = 0
    for k in range(len(bases)):
        our_sum += cosine_similarity(tests[i], bases[k]).sum()
    res_df.loc[i, 'CosMetric'] = our_sum
    

In [16]:
res_df.index = test_df.index


In [17]:
# Среднее значение меры, которое далее берется как порог - относится ли текст к заданной тематике или нет

mean_cos = res_df['CosMetric'].mean()
mean_cos


3.6777702798498595

In [18]:
res_df['Our'] = [1 if i > mean_cos else 0 for i in res_df['CosMetric']]


In [19]:
res_df['Our'].value_counts()


0    2078
1    1882
Name: Our, dtype: int64

In [20]:
res_df.head()


,CosMetric,Our
test_1,1.82601,0
test_10,3.82153,1
test_100,3.70986,1
test_1000,3.69877,1
test_1001,3.75377,1


### Id текстов для валидации результатов (вариант 7)

In [21]:
our_ids = [3078.0, 9.0, 3085.0, 2062.0, 1638.0, 3090.0, 533.0, 534.0, 3607.0, 24.0, 538.0, 30.0, 2225.0,
           3626.0, 46.0, 1074.0, 2100.0, 3639.0, 56.0, 2901.0, 570.0, 2622.0, 3649.0, 3340.0, 581.0, 1098.0,
           2635.0, 1806.0, 1111.0, 89.0, 357.0, 1115.0, 1117.0, 607.0, 358.0, 1634.0, 1126.0, 3178.0, 1131.0,
           2668.0, 110.0, 1647.0, 1214.0, 1137.0, 1653.0, 2167.0, 1113.0, 3892.0, 22.0, 3713.0, 3180.0, 645.0,
           646.0, 3719.0, 2184.0, 536.0, 658.0, 2708.0, 3183.0, 2199.0, 2712.0, 3738.0, 3739.0, 1692.0, 1695.0, 
           160.0, 3745.0, 1188.0, 1704.0, 684.0, 174.0, 1711.0, 3102.0, 1201.0, 1714.0, 1831.0, 2230.0, 2744.0,
           1726.0, 1215.0, 945.0, 3703.0, 3783.0, 3784.0, 1741.0, 1230.0, 720.0, 1747.0, 213.0, 2682.0, 2265.0, 
           3292.0, 742.0, 1767.0, 3818.0, 723.0, 2799.0, 3824.0, 1779.0, 1441.0, 3829.0, 758.0, 3831.0, 3712.0,
           1277.0, 1281.0, 771.0, 262.0, 3850.0, 2316.0, 1294.0, 783.0, 273.0, 274.0, 791.0, 133.0, 3357.0, 1827.0,
           1316.0, 2343.0, 3368.0, 2860.0, 2355.0, 820.0, 2869.0, 311.0, 1849.0, 2688.0, 2878.0, 3904.0, 835.0, 3212.0,
           2885.0, 2374.0, 3913.0, 1361.0, 2386.0, 3925.0, 1594.0, 861.0, 2398.0, 2876.0, 3429.0, 2918.0, 2921.0, 2411.0,
           3184.0, 1524.0, 3959.0, 1406.0, 383.0, 1174.0, 897.0, 2438.0, 1418.0, 1054.0, 1347.0, 399.0, 1006.0, 1425.0, 
           1860.0, 3481.0, 1437.0, 2462.0, 1439.0, 2464.0, 2977.0, 1180.0, 1958.0, 2984.0, 943.0, 2481.0, 3508.0, 2485.0,
           2634.0, 1469.0, 3007.0, 161.0, 2812.0, 3530.0, 462.0, 3538.0, 468.0, 3030.0, 3547.0, 2526.0, 2531.0, 484.0,
           486.0, 2028.0, 2030.0, 498.0, 2548.0, 1013.0, 1586.0, 870.0, 2389.0, 1024.0, 2582.0, 2560.0]


In [22]:
our_ids = ['test_'+ str(int(i)) for i in our_ids]


In [23]:
res_df = res_df[res_df.index.isin(our_ids)]


### Сабмит результатов

In [24]:
defined = []
for i in res_df[res_df['Our'] == 1].index.values:
    defined.append(int(i.split('_')[1]))
    
other = []
for i in res_df[res_df['Our'] == 0].index.values:
    other.append(int(i.split('_')[1]))
    

In [25]:
defined = sorted(defined)
other = sorted(other)


In [26]:
submit = {}

submit['defined'] = defined
submit['other'] = other


In [27]:
with open('lab04.json', 'w') as file:
    json.dump(submit, file)
    